In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import glob

import gzip
import shutil

from config.config_cloud import Config_cloud

In [2]:
root_path = "/mnt/nasbi/no-backups/datasets/video_datasets/cloud_video/"

In [6]:
blur = 9

# Processing training data

In [7]:
cloud_list = glob.glob(root_path+'train/sat/*/*')
cloud_list = np.sort(cloud_list)

In [9]:
data = pd.Series()
data['cloud'] = {}

for cloud_path in cloud_list:
    ID = cloud_path.split('/')[-1].split('.')[0]
    year, month, day, hour, _ = ID.split('-')
    
    cloud = cv2.imread(cloud_path,0)
    cloud = cv2.resize(cloud,(128, 168))
    cloud = cv2.GaussianBlur(cloud,(blur,blur),0)
    cloud = cloud[:160,:]
    
    '''
    plt.figure(figsize=(10,10))
    plt.imshow(cloud,cmap='gray')
    plt.show()
    '''
    data['cloud'][year+month+day+hour] = cloud.astype(np.uint8)
    
data.to_pickle("data/data_train.pkl")

In [6]:
672/4, 512/4

(168.0, 128.0)

In [5]:
np.array([40, 40+420, 130, 130+340])/4

array([ 10. , 115. ,  32.5, 117.5])

In [9]:
np.array([40, 40+420, 130, 130+340])

array([ 40, 460, 130, 470])

In [8]:
cloud.shape

(672, 512)

In [6]:
168/4

42.0

# Processing testing data

In [10]:
cloud_list = glob.glob(root_path+'test/sat/*/*')
cloud_list = np.sort(cloud_list)

In [11]:
data = pd.Series()
data['cloud'] = {}

for cloud_path in cloud_list:
    ID = cloud_path.split('/')[-1].split('.')[0]
    year, month, day, hour, _ = ID.split('-')
    
    cloud = cv2.imread(cloud_path,0)
    cloud = cv2.resize(cloud,(128, 168))
    cloud = cv2.GaussianBlur(cloud,(blur,blur),0)
    cloud = cloud[:160,:]
    '''
    plt.figure(figsize=(10,10))
    plt.imshow(cloud,cmap='gray')
    plt.show()
    '''
    data['cloud'][year+month+day+hour] = cloud.astype(np.uint8)
    
data.to_pickle("data/data_test.pkl")

### Testing list

In [7]:
Cfg = Config_cloud()

In [10]:
from datetime import datetime
data = pd.read_pickle('data/data_test.pkl')

df = pd.read_csv('data/inference_terms.csv')
cloud_time = np.sort(list(data['cloud'].keys())).astype(int)
df.head()

,OpenData_96hr_Start,OpenData_96hr_End,Inference_24hr_Start,Inference_24hr_End,Evaluation_06hr,Evaluation_12hr,Evaluation_18hr,Evaluation_24hr
0,2018/1/2 16:00,2018/1/6 15:00,2018/1/6 16:00,2018/1/7 15:00,2018/1/6 21:00,2018/1/7 3:00,2018/1/7 9:00,2018/1/7 15:00
1,2018/1/9 16:00,2018/1/13 15:00,2018/1/13 16:00,2018/1/14 15:00,2018/1/13 21:00,2018/1/14 3:00,2018/1/14 9:00,2018/1/14 15:00
2,2018/1/16 16:00,2018/1/20 15:00,2018/1/20 16:00,2018/1/21 15:00,2018/1/20 21:00,2018/1/21 3:00,2018/1/21 9:00,2018/1/21 15:00
3,2018/1/23 16:00,2018/1/27 15:00,2018/1/27 16:00,2018/1/28 15:00,2018/1/27 21:00,2018/1/28 3:00,2018/1/28 9:00,2018/1/28 15:00
4,2018/1/30 16:00,2018/2/3 15:00,2018/2/3 16:00,2018/2/4 15:00,2018/2/3 21:00,2018/2/4 3:00,2018/2/4 9:00,2018/2/4 15:00


In [23]:
cloud_clip = []
for i in range(len(df)):
    date_s = df.OpenData_96hr_Start[i]
    date_s = datetime.strptime(date_s, '%Y/%m/%d %H:%M')
    date_s = date_s.strftime('%Y%m%d%H')
    
    date_e = df.OpenData_96hr_End[i]
    date_e = datetime.strptime(date_e, '%Y/%m/%d %H:%M')
    date_e = date_e.strftime('%Y%m%d%H')
    
    s_ind = np.where(cloud_time==int(date_s))[0][0]
    e_ind = np.where(cloud_time==int(date_e))[0][0]
    
    cloud_clip.append(cloud_time[e_ind-23:e_ind+1])

In [25]:
np.save("data/test_cloud_clip.npy", cloud_clip)

In [26]:
cloud_clip

[array([2018010516, 2018010517, 2018010518, 2018010519, 2018010520,
        2018010521, 2018010522, 2018010523, 2018010600, 2018010601,
        2018010602, 2018010603, 2018010604, 2018010605, 2018010606,
        2018010607, 2018010608, 2018010609, 2018010610, 2018010611,
        2018010612, 2018010613, 2018010614, 2018010615]),
 array([2018011216, 2018011217, 2018011218, 2018011219, 2018011220,
        2018011221, 2018011222, 2018011223, 2018011300, 2018011301,
        2018011302, 2018011303, 2018011304, 2018011305, 2018011306,
        2018011307, 2018011308, 2018011309, 2018011310, 2018011311,
        2018011312, 2018011313, 2018011314, 2018011315]),
 array([2018011916, 2018011917, 2018011918, 2018011919, 2018011920,
        2018011921, 2018011922, 2018011923, 2018012000, 2018012001,
        2018012002, 2018012003, 2018012004, 2018012005, 2018012006,
        2018012007, 2018012008, 2018012009, 2018012010, 2018012011,
        2018012012, 2018012013, 2018012014, 2018012015]),
 array([20

### Validating list

In [24]:
Cfg = Config_cloud()
from datetime import datetime
data = pd.read_pickle('data/data_test.pkl')

df = pd.read_csv('data/inference_terms.csv')
cloud_time = np.sort(list(data['cloud'].keys())).astype(int)
df.head()

,OpenData_96hr_Start,OpenData_96hr_End,Inference_24hr_Start,Inference_24hr_End,Evaluation_06hr,Evaluation_12hr,Evaluation_18hr,Evaluation_24hr
0,2018/1/2 16:00,2018/1/6 15:00,2018/1/6 16:00,2018/1/7 15:00,2018/1/6 21:00,2018/1/7 3:00,2018/1/7 9:00,2018/1/7 15:00
1,2018/1/9 16:00,2018/1/13 15:00,2018/1/13 16:00,2018/1/14 15:00,2018/1/13 21:00,2018/1/14 3:00,2018/1/14 9:00,2018/1/14 15:00
2,2018/1/16 16:00,2018/1/20 15:00,2018/1/20 16:00,2018/1/21 15:00,2018/1/20 21:00,2018/1/21 3:00,2018/1/21 9:00,2018/1/21 15:00
3,2018/1/23 16:00,2018/1/27 15:00,2018/1/27 16:00,2018/1/28 15:00,2018/1/27 21:00,2018/1/28 3:00,2018/1/28 9:00,2018/1/28 15:00
4,2018/1/30 16:00,2018/2/3 15:00,2018/2/3 16:00,2018/2/4 15:00,2018/2/3 21:00,2018/2/4 3:00,2018/2/4 9:00,2018/2/4 15:00


In [10]:
cloud_clip = []
for i in range(len(df)):
    date_s = df.OpenData_96hr_Start[i]
    date_s = datetime.strptime(date_s, '%Y/%m/%d %H:%M')
    date_s = date_s.strftime('%Y%m%d%H')
    
    date_e = df.OpenData_96hr_End[i]
    date_e = datetime.strptime(date_e, '%Y/%m/%d %H:%M')
    date_e = date_e.strftime('%Y%m%d%H')
    
    s_ind = np.where(cloud_time==int(date_s))[0][0]
    e_ind = np.where(cloud_time==int(date_e))[0][0]
    
    cloud_clip.append(cloud_time[s_ind:e_ind+1])

np.save("data/valid_cloud_clip.npy", cloud_clip,allow_pickle=True)

In [8]:
cloud_clip

[array([2018010216, 2018010217, 2018010218, 2018010219, 2018010220,
        2018010221, 2018010222, 2018010223, 2018010300, 2018010301,
        2018010302, 2018010303, 2018010304, 2018010305, 2018010306,
        2018010307, 2018010308, 2018010309, 2018010310, 2018010311,
        2018010312, 2018010313, 2018010314, 2018010315, 2018010316,
        2018010317, 2018010318, 2018010319, 2018010320, 2018010321,
        2018010322, 2018010323, 2018010400, 2018010401, 2018010402,
        2018010403, 2018010404, 2018010405, 2018010406, 2018010407,
        2018010408, 2018010409, 2018010410, 2018010411, 2018010412,
        2018010413, 2018010414, 2018010415, 2018010416, 2018010417,
        2018010418, 2018010419, 2018010420, 2018010421, 2018010422,
        2018010423, 2018010500, 2018010501, 2018010502, 2018010503,
        2018010504, 2018010505, 2018010506, 2018010507, 2018010508,
        2018010509, 2018010510, 2018010511, 2018010512, 2018010513,
        2018010514, 2018010515, 2018010516, 2018

In [42]:
import random
import torch


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def generate_valid_set(Cfg):
    valid_fold_list = np.random.choice(50, Cfg.batch_size)#random.sample(range(0, 50), Cfg.batch_size)
    data = pd.read_pickle('data/data_test.pkl')
    data_met = pd.read_pickle('data/data_test_met.pkl')
    data = data['cloud']
    data_met = data_met['HGTs']
    cloud_clip = np.load("data/valid_cloud_clip.npy",allow_pickle=True)
    met_clip = np.load('data/test_meta_clip.npy',allow_pickle=True)

    X = []
    Y = []
    for valid_id in valid_fold_list:
        clip = np.sort(cloud_clip[valid_id])
        start = random.sample(range(0, 47), 1)[0]
        end = start+24
        train_ind = clip[start:end]
        test_ind = clip[end+5:end+24:6]
        
        met_ind = find_nearest(met_clip[valid_id], clip[end])
        #print(clip[end])
        #print(met_ind)
        
        x = []
        y = []
        for i in train_ind:
            x.append(data[str(i)])
        for j in test_ind:
            y.append(data[str(j)])
        x = np.stack(x)/255
        y = np.stack(y)/255
        
        
        x = np.concatenate([x,data_met[str(met_ind)]],axis=0)

        X.append(x)
        Y.append(y)

    X = np.stack(X)
    Y = np.stack(Y)

    X = torch.from_numpy(X).type(torch.float)
    Y = torch.from_numpy(Y).type(torch.float)
    
    return X, Y

In [43]:
X, Y = generate_valid_set(Cfg)

In [46]:
X.max()

tensor(1.)

In [7]:
cloud_clip = np.load("data/valid_cloud_clip.npy",allow_pickle=True)

In [40]:
    data_met = pd.read_pickle('data/data_test_met.pkl')
    data_met = data_met['HGTs']

In [41]:
data_met.keys()

dict_keys(['2018010218', '2018010221', '2018010300', '2018010303', '2018010306', '2018010309', '2018010312', '2018010315', '2018010318', '2018010321', '2018010400', '2018010403', '2018010406', '2018010409', '2018010412', '2018010415', '2018010418', '2018010421', '2018010500', '2018010503', '2018010506', '2018010509', '2018010512', '2018010515', '2018010518', '2018010521', '2018010600', '2018010603', '2018010606', '2018010609', '2018010612', '2018010615', '2018010918', '2018010921', '2018011000', '2018011003', '2018011006', '2018011009', '2018011012', '2018011015', '2018011018', '2018011021', '2018011100', '2018011103', '2018011106', '2018011109', '2018011112', '2018011115', '2018011118', '2018011121', '2018011200', '2018011203', '2018011206', '2018011209', '2018011212', '2018011215', '2018011218', '2018011221', '2018011300', '2018011303', '2018011306', '2018011309', '2018011312', '2018011315', '2018011618', '2018011621', '2018011700', '2018011703', '2018011706', '2018011709', '20180117

In [10]:
end = start+24
train_ind = clip[start:end]
test_ind = clip[end+5:end+24:6]

In [11]:
train_ind

array([2018011815, 2018011816, 2018011817, 2018011818, 2018011819,
       2018011820, 2018011821, 2018011822, 2018011823, 2018011900,
       2018011901, 2018011902, 2018011903, 2018011904, 2018011905,
       2018011906, 2018011907, 2018011908, 2018011909, 2018011910,
       2018011911, 2018011912, 2018011913, 2018011914])

In [12]:
test_ind

array([2018011920, 2018012002, 2018012008, 2018012014])

In [3]:
met_clip = np.load('data/test_meta_clip.npy',allow_pickle=True)

In [6]:
met_clip[0]

array([2018010218, 2018010221, 2018010300, 2018010303, 2018010306,
       2018010309, 2018010312, 2018010315, 2018010318, 2018010321,
       2018010400, 2018010403, 2018010406, 2018010409, 2018010412,
       2018010415, 2018010418, 2018010421, 2018010500, 2018010503,
       2018010506, 2018010509, 2018010512, 2018010515, 2018010518,
       2018010521, 2018010600, 2018010603, 2018010606, 2018010609,
       2018010612, 2018010615])

In [8]:
cloud_clip[0]

array([2018010216, 2018010217, 2018010218, 2018010219, 2018010220,
       2018010221, 2018010222, 2018010223, 2018010300, 2018010301,
       2018010302, 2018010303, 2018010304, 2018010305, 2018010306,
       2018010307, 2018010308, 2018010309, 2018010310, 2018010311,
       2018010312, 2018010313, 2018010314, 2018010315, 2018010316,
       2018010317, 2018010318, 2018010319, 2018010320, 2018010321,
       2018010322, 2018010323, 2018010400, 2018010401, 2018010402,
       2018010403, 2018010404, 2018010405, 2018010406, 2018010407,
       2018010408, 2018010409, 2018010410, 2018010411, 2018010412,
       2018010413, 2018010414, 2018010415, 2018010416, 2018010417,
       2018010418, 2018010419, 2018010420, 2018010421, 2018010422,
       2018010423, 2018010500, 2018010501, 2018010502, 2018010503,
       2018010504, 2018010505, 2018010506, 2018010507, 2018010508,
       2018010509, 2018010510, 2018010511, 2018010512, 2018010513,
       2018010514, 2018010515, 2018010516, 2018010517, 2018010

In [20]:
end = 21
valid_id = 10
clip = cloud_clip[valid_id]
clip[end]

2018032813

In [21]:
find_nearest(met_clip[valid_id], clip[end])

2018032812